In [1]:
%pylab inline
import numpy as np
import EXPtools as et

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


SyntaxError: positional argument follows keyword argument (basis_utils.py, line 253)

In [2]:
# Generate triaxial halo distribution 

In [3]:
npart = int(1e5)
Hern_halo = et.utils.halo.ICHernquist(npart)

In [ ]:
hern_tri = Hern_halo.triaxial([0.5, 0.7, 0.7], rot_angle=30, rot_axis='z')

In [ ]:
plt.scatter(hern_tri[:,1], hern_tri[:,2], s=1)
plt.xlim(-2, 2)
plt.ylim(-2, 2)

In [ ]:
et.basis_builder.basis_utils.